In [1]:

!pip install plotly


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from finta import TA
import mplfinance as mpf
import numpy as np
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import webbrowser
from scipy.signal import find_peaks
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from plotly.tools import mpl_to_plotly
import plotly.express as px
import plotly.tools as tls
import webbrowser

In [2]:
# Đọc dữ liệu từ các tệp Excel
VN2018 = pd.read_excel("2018-Vietnam (1).xlsx")
VN2019 = pd.read_excel("2019-Vietnam (1).xlsx")
VN2020 = pd.read_excel("2020-Vietnam (1).xlsx")
VN2021 = pd.read_excel("2021-Vietnam (1).xlsx")
VN2022 = pd.read_excel("2022-Vietnam (1).xlsx")
# Nối dữ liệu
inf = pd.concat([VN2018, VN2019, VN2020, VN2021, VN2022], ignore_index=True)

# Tiền xử lý dữ liệu
inf = inf.iloc[7:].reset_index(drop=True)
inf.columns = inf.iloc[0]
inf = inf[1:]
inf.reset_index(drop=True, inplace=True)
inf = pd.DataFrame(inf)
# Danh sách các cột không muốn thay đổi
excluded_columns = ["STT", "Mã", "Tên công ty", "Sàn", "Ngành ICB - cấp 1", "Ngành ICB - cấp 2", "Ngành ICB - cấp 3", "Ngành ICB - cấp 4"]

# Kiểm tra và loại bỏ khoảng trắng từ tên cột (ngoại trừ các cột được liệt kê trong excluded_columns)
inf.columns = inf.columns.where(inf.columns.isin(excluded_columns), inf.columns.str.extract(r'\s*([^\n]+)\n', expand=False))

inf.index.name = "Năm"
# Xử lý giá trị NaN
inf = inf.dropna()
inf

,STT,Mã,Tên công ty,Sàn,Ngành ICB - cấp 1,Ngành ICB - cấp 2,Ngành ICB - cấp 3,Ngành ICB - cấp 4,Quý,Năm,...,TM. Chi phí tài chính khác,TM. Chi phí sản xuất theo yếu tố,"TM. Chi phí nguyên liệu, vật liệu",TM. Chi phí nhân công,TM.Chi phí khấu hao tài sản cố định,TM. Chi phí dịch vụ mua ngoài,TM. Chi phí khác bằng tiền,BCTCKH. Doanh thu kế hoạch,BCTCKH. Tổng lợi nhuận kế toán trước thuế,BCTCKH. Lợi nhuận sau thuế thu nhập doanh nghiệp
Năm,,,,,,,,,,,,,,,,,,,,,
1,2,BCM,Becamex IDC,HOSE,Tài chính,Bất động sản,Bất động sản,Bất động sản,5,2018,...,3543121946,9062957842701,1451720418454,1039031680467,223031443148,5870329556428,478844744204,8998000000000,2897000000000,2613000000000
5,6,FPT,FPT Corp,HOSE,Công nghệ Thông tin,Công nghệ Thông tin,Phần mềm & Dịch vụ Máy tính,Phần mềm,5,2018,...,10948936512,20176807407326,3565645393709,7707908345504,1872663628552,5700557333979,1330032705582,26660000000000,4460000000000,3568000000000
6,7,GAS,PV Gas,HOSE,Tiện ích Cộng đồng,"Điện, nước & xăng dầu khí đốt",Nước & Khí đốt,Phân phối xăng dầu & khí đốt,5,2018,...,10091078363,61877448110495,52491143160001,1327486216008,2828502305719,3951895737798,1278420690969,63908000000000,9546000000000,7643000000000
7,8,GVR,Tập đoàn CN Cao su VN,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2018,...,63887590591,0,0,0,0,0,0,24224000000000,5187500000000,4150000000000
9,10,HPG,Hòa Phát,HOSE,Nguyên vật liệu,Tài nguyên Cơ bản,Kim loại,Thép và sản phẩm thép,5,2018,...,16476345182,46701963367260,36585746491178,1584989943790,2259594298268,2639853534460,3631779099564,70000000000000,8375000000000,6700000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8345,1658,XLV,XL và DV Sông Đà,UPCoM,Công nghiệp,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Xây dựng,5,2022,...,0,294033005132,99505172533,59932651810,1514636312,128314960157,4765584320,281818182000,5000000000,4000000000
8346,1659,XMC,Bê tông Xuân Mai,UPCoM,Công nghiệp,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,5,2022,...,0,1938574580316,447648087893,232116235924,64281339949,1135481953301,59046963249,2836926000000,104959000000,77582000000
8347,1660,XMD,Xuân Mai - Đạo Tú,UPCoM,Công nghiệp,Xây dựng và Vật liệu,Xây dựng và Vật liệu,Vật liệu xây dựng & Nội thất,5,2022,...,0,138253205514,105401288692,20349396860,3040011597,7425861340,2036647025,140000000000,4023000000,4023000000


In [3]:
# Lọc dữ liệu theo điều kiện cột "Mã" là "AAA"
inf_aaa = inf[inf["Mã"] == "AAA"].copy()

# In thông tin dữ liệu sau khi lọc
inf_aaa

,STT,Mã,Tên công ty,Sàn,Ngành ICB - cấp 1,Ngành ICB - cấp 2,Ngành ICB - cấp 3,Ngành ICB - cấp 4,Quý,Năm,...,TM. Chi phí tài chính khác,TM. Chi phí sản xuất theo yếu tố,"TM. Chi phí nguyên liệu, vật liệu",TM. Chi phí nhân công,TM.Chi phí khấu hao tài sản cố định,TM. Chi phí dịch vụ mua ngoài,TM. Chi phí khác bằng tiền,BCTCKH. Doanh thu kế hoạch,BCTCKH. Tổng lợi nhuận kế toán trước thuế,BCTCKH. Lợi nhuận sau thuế thu nhập doanh nghiệp
Năm,,,,,,,,,,,,,,,,,,,,,
30,31,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2018,...,35060950805,4303018729044,3425686168227,235972514065,244490805883,361415553980,35453686889,10000000000000,637500000000,510000000000
1702,31,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2019,...,9256505224,4237295639754,3059155241795,345088176730,258634288464,458039973169,116377959596,10000000000000,687500000000,550000000000
3374,31,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2020,...,8882331388,3866397674686,2656949074635,393694037758,286329497570,477616387871,51808676852,9000000000000,687500000000,550000000000
5046,31,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2021,...,17219547009,4991975871175,2668776017829,450002175543,315934993090,1497882808858,59379875855,14100000000000,658750000000,527000000000
6718,31,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",5,2022,...,16237867782,5338577537178,3069898300282,475262677674,311000570200,1421633870086,60782118936,12500000000000,625000000000,500000000000


In [12]:


# Lọc dữ liệu cho mã chứng khoán "AAA"
# Assuming 'inf' is your DataFrame
# Replace 'inf' with the correct DataFrame name if needed
aaa_data = inf[inf['Mã'] == 'AAA']

# Loại bỏ giá trị NaN và Inf
aaa_data = aaa_data.replace([np.inf, -np.inf], np.nan).dropna()

# Calculate investment ratios
aaa_data['Tỷ lệ Đầu tư TSNH'] = (aaa_data['CĐKT. TÀI SẢN NGẮN HẠN'] / aaa_data['CĐKT. TỔNG CỘNG TÀI SẢN']) * 100
aaa_data['Tỷ lệ Đầu tư TSDH'] = (aaa_data['CĐKT. TÀI SẢN DÀI HẠN'] / aaa_data['CĐKT. TỔNG CỘNG TÀI SẢN']) * 100

# Chọn dữ liệu cho biểu đồ hệ số Nợ và hệ số Vốn Chủ sở hữu
aaa_data_selected = aaa_data[(aaa_data['Năm'] >= 2018) & (aaa_data['Năm'] <= 2022)]
aaa_data_selected['Hệ số Nợ'] = (aaa_data_selected['CĐKT. NỢ PHẢI TRẢ'] / aaa_data_selected['CĐKT. TỔNG CỘNG TÀI SẢN']) * 100
aaa_data_selected['Hệ số Vốn Chủ sở hữu'] = (aaa_data_selected['CĐKT. VỐN CHỦ SỞ HỮU'] / aaa_data_selected['CĐKT. TỔNG CỘNG TÀI SẢN']) * 100

# Tính toán Net Working Capital
aaa_data_selected['Net Working Capital'] = aaa_data_selected['CĐKT. TÀI SẢN NGẮN HẠN'] - aaa_data_selected['CĐKT. Nợ ngắn hạn']

# Chọn chỉ các năm 2018, 2019, 2020, 2021, và 2022
selected_years = aaa_data[aaa_data['Năm'].isin([2018, 2019, 2020, 2021, 2022])]

# Tính tỷ số nợ trên tổng tài sản và tỷ số nợ vay trên tổng tài sản
selected_years['Tỷ số nợ'] = selected_years['CĐKT. NỢ PHẢI TRẢ'] / selected_years['CĐKT. TỔNG CỘNG TÀI SẢN'].replace(0, np.nan)
selected_years['Tỷ số nợ vay'] = (selected_years['CĐKT. Vay và nợ thuê tài chính ngắn hạn'] + selected_years['CĐKT. Vay và nợ thuê tài chính dài hạn']) / selected_years['CĐKT. TỔNG CỘNG TÀI SẢN'].replace(0, np.nan)

# Loại bỏ các giá trị NaN và vô cùng lớn/vô cùng nhỏ
selected_years = selected_years.replace([np.inf, -np.inf], np.nan).dropna()

# Chuyển đồ thành Dash app
app = dash.Dash(__name__)

# Layout của ứng dụng
app.layout = html.Div(children=[
    html.H1(children='AAA Financial Data'),

    dcc.Dropdown(
        id='chart-dropdown',
        options=[
            {'label': 'Biểu đồ cột tỷ lệ đầu tư', 'value': 'investment-bar-chart'},
            {'label': 'Biểu đồ đường lưu chuyển tiền tệ', 'value': 'cashflow-line-chart'},
            {'label': 'Biểu đồ hệ số Nợ và hệ số Vốn Chủ sở hữu', 'value': 'debt-equity-ratio-bar-chart'},
            {'label': 'Biểu đồ đường Net Working Capital', 'value': 'net-working-capital-line-chart'},
            {'label': 'Biểu đồ cấu trúc tài sản', 'value': 'asset-structure-line-chart'},
            {'label': 'Biểu đồ hệ số khả năng thanh toán', 'value': 'custom-matplotlib-chart'},
            {'label': 'Biểu đồ hệ số thanh toán ngắn hạn', 'value': 'short-term-liquidity-chart'},
        ],
        value='investment-bar-chart'  # Giá trị mặc định cho dropdown menu
    ),

    dcc.Graph(id='selected-chart')
])

# Callback để cập nhật biểu đồ theo lựa chọn từ dropdown menu
@app.callback(
    Output('selected-chart', 'figure'),
    [Input('chart-dropdown', 'value')]
)
def update_chart(selected_chart):
    try:
        if selected_chart == 'investment-bar-chart':
            # Biểu đồ cột tỷ lệ đầu tư
            fig = px.bar(aaa_data, x='Năm', y=['Tỷ lệ Đầu tư TSNH', 'Tỷ lệ Đầu tư TSDH'],
                         labels={'value': 'Tỷ lệ đầu tư', 'variable': 'Loại đầu tư'},
                         title='Tỷ lệ đầu tư tài sản của AAA từ 2018 đến 2022',
                         color_discrete_map={'Tỷ lệ Đầu tư TSNH': '#0072BC', 'Tỷ lệ Đầu tư TSDH': '#8DC63F'},
                         height=600)

            fig.update_layout(yaxis_title='Tỷ lệ (%)', yaxis_range=[0, 100])

            return fig
        elif selected_chart == 'cashflow-line-chart':
            # Biểu đồ đường lưu chuyển tiền tệ
            fig = px.line(aaa_data, x='Năm', y=['LCTT. Lưu chuyển tiền tệ ròng từ các hoạt động sản xuất kinh doanh (TT)',
                                               'LCTT. Lưu chuyển tiền tệ ròng từ hoạt động đầu tư (TT)',
                                               'LCTT. Lưu chuyển tiền tệ từ hoạt động tài chính (TT)'],
                          labels={'value': 'Lưu chuyển tiền tệ ròng', 'variable': 'Loại hoạt động'},
                          title='Lưu chuyển tiền tệ ròng từ các hoạt động của AAA qua các năm (2018-2022)')

            return fig
        elif selected_chart == 'debt-equity-ratio-bar-chart':
            # Biểu đồ cột chồng lên nhau cho Hệ số Nợ và Hệ số Vốn Chủ sở hữu bằng Plotly Express
            fig = px.bar(aaa_data_selected, x='Năm', y=['Hệ số Nợ', 'Hệ số Vốn Chủ sở hữu'],
                         labels={'variable': 'Loại hệ số'},
                         title='Biểu đồ hệ số Nợ và hệ số Vốn Chủ sở hữu của AAA từ 2018 đến 2022',
                         color_discrete_map={'Hệ số Nợ': '#0072BC', 'Hệ số Vốn Chủ sở hữu': '#8DC63F'},
                         height=600)

            fig.update_layout(barmode='stack', yaxis_title='Tỷ lệ (%)', yaxis_range=[0, 100])

            return fig
        elif selected_chart == 'net-working-capital-line-chart':
            # Biểu đồ đường Net Working Capital
            fig = px.line(aaa_data_selected, x='Năm', y='Net Working Capital',
                          labels={'Net Working Capital': 'Net Working Capital'},
                          title='Biểu đồ Net Working Capital của AAA từ 2018 đến 2022')

            return fig
        elif selected_chart == 'asset-structure-line-chart':
            # Biểu đồ đường cấu trúc tài sản
            fig = px.line(selected_years, x='Năm', y=['Tỷ số nợ', 'Tỷ số nợ vay'],
                          labels={'value': 'Tỷ số', 'variable': 'Loại'},
                          title='Cấu trúc tài sản của AAA qua các năm (2018-2022)',
                          color_discrete_map={'Tỷ số nợ': '#FF5733', 'Tỷ số nợ vay': '#00537E'})

            return fig
        elif selected_chart == 'custom-matplotlib-chart':
            # Tính hệ số thanh khoản
            aaa_data_selected['Hệ số thanh khoản'] = (
                aaa_data_selected['CĐKT. TỔNG CỘNG TÀI SẢN'] /
                aaa_data_selected['CĐKT. NỢ PHẢI TRẢ']
            )

            # Tạo hình Matplotlib
            fig_matplotlib, ax = plt.subplots(figsize=(10, 6))
            ax.plot(aaa_data_selected['Năm'], aaa_data_selected['Hệ số thanh khoản'], marker='o', color='#8DC63F', label='Hệ số thanh khoản')
            ax.set_title('Hệ số thanh khoản của AAA từ 2018 đến 2022')
            ax.set_xlabel('Năm')
            ax.set_ylabel('Hệ số thanh khoản')
            ax.grid(True)

            # Đặt các dấu chia x trên trục x để hiển thị các năm cụ thể
            years = [2018, 2019, 2020, 2021, 2022]
            ax.set_xticks(years)
            ax.set_xticklabels([f'Năm {year}' for year in years])

            # Thêm chú thích
            ax.legend()

            # Chuyển đổi hình Matplotlib sang định dạng Plotly
            mpl_fig = tls.mpl_to_plotly(fig_matplotlib)

            return mpl_fig
        elif selected_chart == 'short-term-liquidity-chart':
            # Tính toán hệ số khả năng thanh toán ngắn hạn
            aaa_data['Hệ số thanh toán ngắn hạn'] = (
                aaa_data['CĐKT. TÀI SẢN NGẮN HẠN'] /
                aaa_data['CĐKT. Nợ ngắn hạn']
            )

            # Biểu đồ cột cho hệ số khả năng thanh toán ngắn hạn
            fig = go.Figure()
            fig.add_trace(go.Bar(x=aaa_data['Năm'], y=aaa_data['Hệ số thanh toán ngắn hạn'], marker_color='#0072BC', name='Hệ số thanh toán ngắn hạn'))
            fig.update_layout(title='Biểu đồ hệ số thanh toán ngắn hạn của AAA từ 2018 đến 2022',
                              xaxis_title='Năm', yaxis_title='Hệ số thanh toán ngắn hạn')

            return fig

    except Exception as e:
        print(f"Lỗi cập nhật biểu đồ: {str(e)}")
        return None

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
    webbrowser.open_new("http://localhost:8051")
